<a href="https://colab.research.google.com/github/siriusted/gym-dssat-notebooks/blob/master/First_steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First steps with Gym-dssat

In this tutorial, we will show how to simply interact with the gym-dssat environment.

First, we need to install gym-dssat. This is done in the following code cell. 

**Note**: It might take a while

## Installation

In [ ]:
!wget https://raw.githubusercontent.com/siriusted/gym-dssat-notebooks/master/install.sh
!chmod u+x install.sh
!./install.sh

Now, we're all set up to use gym-dssat!
<br/><br/>
To be able use gym-dssat properly, we need to run commands using `pdirun`. As a result, the source code for the rest of the tutorial has been collected into a script, which we will fetch next, then run.


In [ ]:
!wget https://raw.githubusercontent.com/siriusted/gym-dssat-notebooks/master/first_steps.py

Run example. You can take a look at `first_steps.py` in the file browser here on colab

In [ ]:
!/opt/pdi/bin/pdirun python first_steps.py

The results of the script have been written into `results.pkl`. 

Next, we will load and display the results.

In [ ]:
import pickle
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def plot_results(data):
    data_dict = {}
    for label, returns in data:
        data_dict[label] = returns
    df = pd.DataFrame(data_dict)
    
    ax = sns.boxplot(data=df)
    ax.set_xlabel("policy")
    ax.set_ylabel("evaluation output")
    plt.show()

with open("results.pkl", "rb") as result_file:
    results = pickle.load(result_file)

In [ ]:
plot_results(results)

And, there you have it! Now, you can go ahead and keep playing with gym-dssat by editing `first_steps.py` and re-running the code cell above.